In [1]:
# %load_ext autoreload
# %autoreload 2
import models, utils
import orca

In [2]:
orca.run([
    "rsh_simulate",              # residential sales hedonic
    "nrh_simulate",              # non-residential rent hedonic

    "households_relocation",     # households relocation model
    "hlcm_simulate",            # households location choice
    "households_transition",     # households transition

    "jobs_relocation",           # jobs relocation model
    "elcm_simulate",             # employment location choice
    "jobs_transition",           # jobs transition

    "feasibility",               # compute development feasibility
    "residential_developer",     # build residential buildings
    "non_residential_developer", # build non-residential buildings
], iter_vars=[2010])

Running iteration 1 with iteration value 2010
Running step 'rsh_simulate'
Filling column residential_units with value 0 (0 values)
Filling column non_residential_rent with value 0 (142400 values)
Filling column non_residential_sqft with value 0 (1341 values)
Filling column year_built with value 1927.0 (3116 values)
Filling column residential_sales_price with value 0 (14196 values)
Filling column building_type_id with value 2.0 (0 values)
count    1.407800e+05
mean     1.156971e+03
std      4.150325e+04
min      2.067312e-03
25%      2.527275e+02
50%      3.799935e+02
75%      5.542447e+02
max      1.250703e+07
dtype: float64
Time to execute step 'rsh_simulate': 1.02 s
Running step 'nrh_simulate'
Filling column job_category with value service (331 values)
count    10803.000000
mean        20.729909
std          6.833292
min          6.258496
25%         17.431477
50%         20.908725
75%         25.077906
max         47.266957
dtype: float64
Time to execute step 'nrh_simulate': 1.16 s


C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\developer\sqftproforma.py:394: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ave_cost_sqft'][c.fars > c.max_industrial_height] = np.nan
C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\developer\sqftproforma.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ave_cost_sqft'][c.fars > c.max_retail_height] = np.nan


Describe of the yearly rent by use
              retail    industrial         office    residential
count  152205.000000  94493.000000  152259.000000  153228.000000
mean       23.867557      8.650493      21.274562      19.306885
std         3.553082      1.641869       2.979941       5.179699
min        18.000000      6.000000      16.000000       1.150000
25%              NaN           NaN            NaN            NaN
50%              NaN           NaN            NaN            NaN
75%              NaN           NaN            NaN            NaN
max        35.000000     14.500000      31.000000      40.550000
Computing feasibility for form industrial
Computing feasibility for form office


C:\Users\Harrison Freund\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


Computing feasibility for form residential
Computing feasibility for form mixedresidential
Computing feasibility for form mixedoffice
Computing feasibility for form retail
Time to execute step 'feasibility': 3.38 s
Running step 'residential_developer'
Number of agents: 362,867
Number of agent spaces: 374,133
Current vacancy = 0.03
Target vacancy = 0.15, target of new units = 52,769
4,217 feasible buildings before running developer
Sum of net units that are profitable: 45,822
WARNING THERE WERE NOT ENOUGH PROFITABLE UNITS TO MATCH DEMAND
Adding 2,557 buildings with 52,280 residential_units
1,660 feasible buildings after running developer
Time to execute step 'residential_developer': 0.37 s
Running step 'non_residential_developer'
Number of agents: 236,711
Number of agent spaces: 642,004
Current vacancy = 0.63
Target vacancy = 0.15, target of new units = 0
1,660 feasible buildings before running developer
Sum of net units that are profitable: 95,189
Adding 0 buildings with 0 job_spaces
1